<a href="https://colab.research.google.com/github/mohamedSalehMatar/Pre-Processing-of-Steel-Factory-Power-Consumption/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
datatest= pd.read_csv('/content/gdrive/MyDrive/DataScience/ML Project/test.csv')
#print(data1)

In [ ]:
datatrain= pd.read_csv('/content/gdrive/MyDrive/DataScience/ML Project/train.csv')
#print(data2)

Task Management:

Mohamed Saleh -> Data Preprocessing ✔️

Mayar         -> Modeling

Arwaa         -> Fine tuning

---------------------------------------------------------------

Problems of this dataset:

1- WeekStatus, Day_of_week and Load_Type are all categorical ✔️
  - Load_Type -> label encoder
  - WeekStatus -> one hot
  - Day_of_week -> one hot

2- missing numerical values ✔️

3- date ✔️

4- filling outliers with mean ✔️

5- duplicates ✔️

6- (potential problems)

In [ ]:
#transforming date to dt type
datatrain['date'] = pd.to_datetime(datatrain['date'])

#creating new columns for each date data
datatrain['year'] = datatrain['date'].dt.year
datatrain['month'] = datatrain['date'].dt.month
datatrain['day'] = datatrain['date'].dt.day

#droping the old date column and reordering the dataframe
datatrain = datatrain.drop(datatrain.columns[1], axis=1)
neworder = [
'Id'
,'year'
,'month'
,'day'
,'Lagging_Current_Reactive.Power_kVarh'
,'Leading_Current_Reactive_Power_kVarh'
,'CO2(tCO2)'
,'Lagging_Current_Power_Factor'
,'Leading_Current_Power_Factor'
,'NSM'
,'WeekStatus'
,'Day_of_week'
,'Load_Type'
,'Usage_kWh'
]
datatrain = datatrain[neworder]

#print(datatrain)

In [ ]:
sns.heatmap(datatrain.corr(), cmap='RdBu', vmin=-1, vmax=1, annot=True)

In [ ]:
datatrain.duplicated().sum()

In [ ]:
#data null-value amout
datatrain.isnull().sum()

In [ ]:
#filling categorical null-value with mode

datatrain.iloc[:, 10:12] = datatrain.iloc[:, 10:12].fillna(datatrain.iloc[:, 10:12].mode().iloc[0])
datatrain.isnull().sum()

In [ ]:
numerical_features = datatrain.select_dtypes(include=['float', 'int']).columns

for feature in numerical_features:
    print(f'info of {feature} = ',datatrain[feature].describe())
    Q1 = np.percentile(datatrain[feature], 25, method='midpoint')
    Q3 = np.percentile(datatrain[feature], 75, method='midpoint')
    IQR = Q3 - Q1
    print(f'\nIQR of {feature} = ', IQR)

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print(f'Lower bound of {feature} = ', lower)
    print(f'Upper bound of {feature} = ', upper)

    # Replace the outliers with the feature mean
    datatrain.loc[(datatrain[feature] < lower) | (datatrain[feature] > upper), feature] = None

    print(f'\nOutliers of {feature} is no more.')

In [ ]:
datatrain.isnull().sum()

In [ ]:
#filling numerical null-value with mean

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(datatrain.iloc[:, [4, 5, 6, 8, 13]])
datatrain.iloc[:, [4, 5, 6, 8, 13]] = imputer.transform(datatrain.iloc[:, [4, 5, 6, 8, 13]])
datatrain.isnull().sum()

In [ ]:
#transforming categorical date into numeriacal ones

label_encoder = LabelEncoder()

# Fit and transform the data
encoded_load_types = label_encoder.fit_transform(datatrain['Load_Type'])
datatrain['Load_Type'] = encoded_load_types


In [ ]:
datatrain = pd.get_dummies(datatrain, columns = ['WeekStatus', 'Day_of_week'])

column_to_move = datatrain.pop("Usage_kWh")
datatrain.insert(len(datatrain.columns), "Usage_kWh", column_to_move)

In [ ]:
datatrain.head(10)

In [ ]:
datatrain.tail(10)